Behind the curtain, what Dask dataframes do is to coordinate many Pandas dataframes, partitioned along an index. The figure below demonstrates this:

<img src="assets/dask_dataframe.svg" width="30%" alt="Dask dataframes are blocked Pandas dataframes">

In doing so, they support a large subset of the Pandas api and hence we use them almost as if we use Pandas dataframes. This is something admirable. Big data world has a lot of technologies that use their own apis and syntax. Learning them is a main obstacle in adopting these technologies. Dask's approach in this respect is something welcomed as many data scientists are already familiar with the Pandas.

**Note that we'll be using Dask installed on our local machines not on a cluster**. In doing so, we'll use multiple CPU cores of our machines to parallelize the dataframe operations. Remember that all the codes we'll write here should also run in a clustered environment with a proper configuration.

# Working with Dask dataframes

We'll start using Dask dataframes by loading a csv formatted file. But before that, we start a Dask client. As we mentioned in the previous checkpoint, starting a client isn't necessary when working with Dask on our local computer but still it provides us a dashboard where we can investigate our computations.

The link to the dashboard will become visible when you create the client below. We suggest opening the dashboard and observing what happens when running the Dask code. It's a good way of learning what is going on.

In [1]:
import warnings
warnings.filterwarnings("ignore")

from dask.distributed import Client, progress

client = Client(n_workers=4, threads_per_worker=2, memory_limit='2GB')
client

Client Scheduler: tcp://127.0.0.1:50164 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 8.00 GB


# Loading data into Dask dataframes

We first load a dataset that's relatively large on a Dask dataframe. The dataset we'll use is the [Credit Card Fraud Detection](https://www.kaggle.com/mlg-ulb/creditcardfraud) dataset from Kaggle. It contains financial transactions along with an identifier whether a transaction is fraud or not. The dataset includes several features which are the principal components of the original data. Since the data is confidential, the authors of the data prefers to put only the principal components instead of the real variables. Since our focus here is to demonstrate the capabilities of Dask dataframes, this isn't something problematic for our purposes.

We start by importing the Dask dataframes as:

```python
import dask.dataframe as dd
```

Then we load the csv file using the `.read_csv()` method of Dask dataframe. Notice that this is the identically named counterpart of the `.read_csv()` method of the Pandas dataframe:

In [4]:
# Dataframes implement the Pandas API
import dask.dataframe as dd

# This loads the data into Dask dataframe
df = dd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/creditcard.csv')

When working with Dask, we should always be aware of the fact that Dask objects are evaluated lazily. That is to say, unlike Pandas, when we load a dataset to a Dask dataframe, the data will not be available until we call `.compute()` function. For example, the code below would return just the names of the columns and their data types:

In [5]:
print(df)

Dask DataFrame Structure:
                Time       V1       V2       V3       V4       V5       V6       V7       V8       V9      V10      V11      V12      V13      V14      V15      V16      V17      V18      V19      V20      V21      V22      V23      V24      V25      V26      V27      V28   Amount  Class
npartitions=3                                                                                                                                                                                                                                                                                   
               int64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  int64
                 ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      

We can also see the data types of the columns by calling the `.dtypes` attribute:

In [6]:
df.dtypes

Time        int64
V1        float64
V2        float64
V3        float64
V4        float64
V5        float64
V6        float64
V7        float64
V8        float64
V9        float64
V10       float64
V11       float64
V12       float64
V13       float64
V14       float64
V15       float64
V16       float64
V17       float64
V18       float64
V19       float64
V20       float64
V21       float64
V22       float64
V23       float64
V24       float64
V25       float64
V26       float64
V27       float64
V28       float64
Amount    float64
Class       int64
dtype: object

Dask dataframes provide many similar methods to Pandas dataframes. As an example, let's call the `.head()` function and see what it returns:

In [7]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


We observe some data as the result of the `.head()` function. Then, what about the lazy evaluation? The reason is that some operations on Dask dataframes will automatically display the data, yet some operations do not. For example, the `.describe()` method would return nothing:

In [8]:
df.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
npartitions=1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


Instead, we should call `.compute()` to evaluate it as follows:

In [9]:
df.describe().compute()

ValueError: Mismatched dtypes found in `pd.read_csv`/`pd.read_table`.

+--------+---------+----------+
| Column | Found   | Expected |
+--------+---------+----------+
| Time   | float64 | int64    |
+--------+---------+----------+

Usually this is due to dask's dtype inference failing, and
*may* be fixed by specifying dtypes manually by adding:

dtype={'Time': 'float64'}

to the call to `read_csv`/`read_table`.

Alternatively, provide `assume_missing=True` to interpret
all unspecified integer columns as floats.

Oops! We got an error. That's because Dask tried to infer the data types of the columns it loads. However, Dask doesn't consider all the values in each column when deciding the data types due to the lazy evaluation. Instead, it randomly sample a subset of the columns and guess what their data types are. In case we encounter these type of situations, we need to specify the column types manually. Moreover, setting the `assume_missing` parameter of the `read_csv()` function to `True` would also be helpful in some situations as it informs Dask that all integer columns that aren’t specified in `dtype` parameter are assumed to contain missing values, and are converted to floats.

Now, let's read our dataset again but specify the type of the `Time` columns as float64:

In [10]:
df = dd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/creditcard.csv', dtype={'Time': 'float64'})

We can now run the `.describe()` method and `.compute()` to see the descriptive statistics:

In [11]:
df.describe().compute()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,6.003539e-16,6.386744e-18,-3.078411e-15,2.484443e-15,3.832046e-16,1.542399e-15,-1.341216e-15,1.532819e-16,-1.362372e-15,...,3.512709e-17,1.119277e-15,3.153455e-16,4.394479e-15,1.079360e-15,1.743781e-15,-3.506722e-16,-1.003916e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,49346.000000,-7.868600e-01,-5.536365e-01,-7.156553e-01,-7.067841e-01,-4.093779e-01,-6.546632e-01,-4.787135e-01,-1.339191e-01,-5.477089e-01,...,-2.256367e-01,-5.250530e-01,-1.301482e-01,-3.241623e-01,-2.218416e-01,-2.821043e-01,-6.089022e-02,-2.762231e-02,6.840000,0.000000
50%,94872.000000,9.301532e-02,8.999754e-02,1.743346e-01,1.809941e-01,1.480477e-01,-1.649056e-01,1.590763e-01,7.790153e-02,2.583745e-02,...,6.636310e-03,1.241855e-01,5.160336e-02,6.794774e-02,1.674311e-01,-1.290092e-03,1.088230e-02,2.345040e-02,24.990000,0.000000
75%,165830.000000,1.984826e+00,8.916958e-01,1.373682e+00,1.012508e+00,8.613822e-01,4.775488e-01,7.334868e-01,3.706835e-01,7.092123e-01,...,2.385322e-01,7.308589e-01,2.346538e-01,5.274699e-01,4.190202e-01,2.938470e-01,1.060637e-01,8.061929e-02,84.910000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


Voila!

# Using Dask as if it's Pandas

Next, we demonstrate how we can do some commonly used Pandas dataframe operations on Dask dataframes. In a nutshell, most of the common Pandas operations operate almost identically on Dask dataframes. The major difference is to call `.compute()` to evaluate the result when doing operations on Dask dataframes.

## Filtering the data

One of the most common scenarios when using Pandas is to do filtering on a dataframe. Let's filter our Dask dataframe. Specifically, we want to filter only those transactions that are larger than 10000 US dollars:

In [12]:
df2 = df[df["Amount"] > 10000]
df2

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
npartitions=3,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


As usual, nothing returned. This is because we need to call `.compute()`:

In [13]:
df[df["Amount"] > 10000].compute()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
46841,42951.0,-23.712839,-42.172688,-13.320825,9.925019,-13.945538,5.564891,15.710644,-2.844253,-1.580725,...,7.921600,-6.320710,-11.310338,0.404175,-4.547278,-1.577118,-2.357385,2.253662,12910.93,0
54018,46253.0,-21.780665,-38.305310,-12.122469,9.752791,-12.880794,4.256017,14.785051,-2.818253,-0.667338,...,7.437478,-5.619439,-10.547038,0.653249,-4.232409,-0.480459,-2.257913,2.082488,11898.09,0
58465,48401.0,-36.802320,-63.344698,-20.645794,16.715537,-20.672064,7.694002,24.956587,-4.730111,-2.687312,...,11.455313,-10.933144,-17.173665,1.180700,-7.025783,-2.534330,-3.602479,3.450224,19656.53,0
30325,95286.0,-34.549296,-60.464618,-21.340854,16.875344,-19.229075,6.335259,24.422716,-4.964566,0.188912,...,11.502580,-9.499423,-16.513186,0.744341,-7.081325,-2.604551,-3.550963,3.250802,18910.00,0
48486,119713.0,-20.924897,-37.943452,-14.060281,10.473005,-10.866639,6.256654,14.960521,-2.392155,-0.597076,...,6.829810,-6.926353,-9.928657,-0.447084,-4.848151,-2.241620,-2.140723,2.001492,11789.84,0
32983,166198.0,-35.548539,-31.850484,-48.325589,15.304184,-113.743307,73.301626,120.589494,-27.347360,-3.872425,...,-21.620120,5.712303,-1.581098,4.584549,4.554683,3.415636,31.612198,-15.430084,25691.16,0
42461,172273.0,-9.030538,-11.112584,-16.233798,3.592021,-40.427726,23.917837,44.054461,-7.277778,-4.210637,...,-0.269048,0.988144,7.040028,0.347693,2.520869,2.342495,3.478175,-2.713136,10199.44,0


If you were to look at the Dask dashboard during the above computation, you would see something like the following on the "Workers" tab:

![dask dashboard](assets/dask_dashboard.png)

We can also use the `.loc` and `.iloc` functions on Dask dataframes. Again, we shouldn't forget to call the `.compute()` method:

In [14]:
df.loc[df["Amount"] > 10000,:].compute()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
46841,42951.0,-23.712839,-42.172688,-13.320825,9.925019,-13.945538,5.564891,15.710644,-2.844253,-1.580725,...,7.921600,-6.320710,-11.310338,0.404175,-4.547278,-1.577118,-2.357385,2.253662,12910.93,0
54018,46253.0,-21.780665,-38.305310,-12.122469,9.752791,-12.880794,4.256017,14.785051,-2.818253,-0.667338,...,7.437478,-5.619439,-10.547038,0.653249,-4.232409,-0.480459,-2.257913,2.082488,11898.09,0
58465,48401.0,-36.802320,-63.344698,-20.645794,16.715537,-20.672064,7.694002,24.956587,-4.730111,-2.687312,...,11.455313,-10.933144,-17.173665,1.180700,-7.025783,-2.534330,-3.602479,3.450224,19656.53,0
30325,95286.0,-34.549296,-60.464618,-21.340854,16.875344,-19.229075,6.335259,24.422716,-4.964566,0.188912,...,11.502580,-9.499423,-16.513186,0.744341,-7.081325,-2.604551,-3.550963,3.250802,18910.00,0
48486,119713.0,-20.924897,-37.943452,-14.060281,10.473005,-10.866639,6.256654,14.960521,-2.392155,-0.597076,...,6.829810,-6.926353,-9.928657,-0.447084,-4.848151,-2.241620,-2.140723,2.001492,11789.84,0
32983,166198.0,-35.548539,-31.850484,-48.325589,15.304184,-113.743307,73.301626,120.589494,-27.347360,-3.872425,...,-21.620120,5.712303,-1.581098,4.584549,4.554683,3.415636,31.612198,-15.430084,25691.16,0
42461,172273.0,-9.030538,-11.112584,-16.233798,3.592021,-40.427726,23.917837,44.054461,-7.277778,-4.210637,...,-0.269048,0.988144,7.040028,0.347693,2.520869,2.342495,3.478175,-2.713136,10199.44,0


## Aggregating with group by

Another common operation on dataframes is to aggregate some columns after applying group by on a categorical column. To demonstrate this, let's group by our dataframe with respect to the  `Class` column and see whether the average `Amount` differs between frauds and legal transactions:

In [15]:
df2 = df.groupby("Class")["Amount"].mean().compute()
df2

Class
0     88.291022
1    122.211321
Name: Amount, dtype: float64

It seems that fraud transactions are higher than the other on average.

## Using apply() function

A good use case for parallel processing is to use the `.apply()` function. If we use `.apply()` on a column to operate value by value, distributing the workload across many cores would speed up the calculations. As a toy example, say that we want to transform the `Amount` column such that amounts that are higher than 10000 US dollars will be 1 and the amounts that are lower than and equal to 10000 US dollars will be 0:

In [16]:
df["Amount2"] = df["Amount"].apply(lambda x: 1 if x>10000 else 0, meta=('Amount2', 'int64'))
df["Amount2"].head(20)

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
Name: Amount2, dtype: int64

This type of operations lay itself to parallelization well, because in the example above, the `.apply()` function operates row by row and each operation is independent to each other. Hence, rows can be distributed over several cores or even machines.

# Persisting data in memory

So far, we saw that we can load large amounts of data into Dask dataframes and do parallel computations on them. However, Dask can also be convenient even if we have a sufficient amount of RAM in our computer. If we have the available memory for a dataset, then we can persist the data in memory and take advantage of the memory speed. Dask still will be useful when parallelizing the operations. If we do this, all of the future computations on the persisted dataframe will be much faster.

In order to persist a dataframe to memory, we just call `.persist()` method of the Dask dataframes:

In [17]:
df = df.persist()

# Drawbacks of Dask dataframe

As we saw in this checkpoint, working with Dask dataframes is very similar to working with Pandas. That being said, we should be aware of the fact that `Dask.dataframe` package only covers a small but well-used portion of the Pandas api. This because of the following two reasons:

1.  The Pandas api is *huge* and Dask still has a way to go in order to increase its coverage.
2.  Some operations are hard to do in parallel like sorting. Hence, not all of the functionalities of Pandas will be on the Dask dataframes.

Additionally, some important operations like ``set_index`` work, but it works slower than Pandas because sorting includes substantial shuffling of the data. Since Dask stores the data in a Dask dataframe in several Pandas dataframes, sometimes it needs to write out to disk in order the coordinate the sorting operation. This causes an overhead.

# Closing the connection

We're done with the Dask for this checkpoint. So, we can close our connection:

In [15]:
client.close()